Connection with drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd
import os
import librosa
import numpy as np
from tqdm import tqdm
!pip install llvmlite==0.31.0
!pip install resampy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for llvmlite
  Running setup.py clean for llvmlite
Failed to build llvmlite
ERROR: Could not build wheels for llvmlite, which is required to install pyproject.toml-based projects
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 34.5 MB/s eta 0:00:00


# Feature extr

In [ ]:
#### Extracting MFCC's For every audio file
audio_dataset_path='/content/drive/MyDrive/final project dataset/audio'
metadata=pd.read_csv('/content/drive/MyDrive/final project dataset/metadata/3ragafinal01.csv')

In [ ]:
def features_extractor(file):
    audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    
    return mfccs_scaled_features
    

In [ ]:
### Now we iterate through every audio file and extract features 
### using Mel-Frequency Cepstral Coefficients
extracted_features=[]
for index_num,row in tqdm(metadata.iterrows()):
    file_name = os.path.join(os.path.abspath(audio_dataset_path),str(row["fold"])+'/',str(row["file_name"]))
    final_class_labels=row["class"]
    data=features_extractor(file_name)
    extracted_features.append([data,final_class_labels])

246it [02:34,  1.89it/s]/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:256: UserWarning: n_fft=2048 is too large for input signal of length=370
  warnings.warn(
252it [02:36,  2.07it/s]/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:256: UserWarning: n_fft=2048 is too large for input signal of length=903
  warnings.warn(
371it [03:45,  1.65it/s]


In [ ]:
### converting extracted_features to Pandas dataframe
extracted_features_df=pd.DataFrame(extracted_features,columns=['feature','class'])
extracted_features_df.head()

,feature,class
0,"[-163.36482, 153.0693, -75.534676, 49.750607, ...",chakravaakam
1,"[-150.46718, 153.124, -78.676575, 48.29218, -0...",chakravaakam
2,"[-146.13354, 151.12587, -76.37766, 51.304493, ...",chakravaakam
3,"[-151.48657, 151.21315, -81.44583, 47.865356, ...",chakravaakam
4,"[-154.61449, 159.91881, -73.21971, 49.904778, ...",chakravaakam


In [ ]:
### Split the dataset into independent and dependent dataset
x=np.array(extracted_features_df['feature'].tolist())
y=np.array(extracted_features_df['class'].tolist())


In [ ]:
a=x
b=y

In [ ]:
### Label Encoding
###y=np.array(pd.get_dummies(y))
### Label Encoder
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(y))

In [ ]:
## Train Test Split

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

**model** **creation**

In [ ]:
#model creation


import tensorflow
# print(tf.__version__)

In [ ]:
from sklearn import metrics
from tensorflow.python.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.python.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [ ]:
### No of classes
num_labels=y.shape[1]

In [ ]:
model=Sequential()
###first layer
model.add(Dense(100,input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###second layer
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###third layer
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.5))

###final layer
model.add(Dense(num_labels))
model.add(Activation('softmax'))

In [ ]:
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               4100      
_________________________________________________________________
activation (Activation)      (None, 100)               0         
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 200)               20200     
_________________________________________________________________
activation_1 (Activation)    (None, 200)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               2

In [ ]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')


In [ ]:
## Training my model
from tensorflow import keras
from tensorflow.python.keras.callbacks import ModelCheckpoint
from datetime import datetime 

num_epochs = 100
num_batch_size = 32
checkpointer = ModelCheckpoint(filepath='saved_models/audio_classification.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

  model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs,
            validation_data=(x_test, y_test), callbacks=[checkpointer], verbose=1)

duration = datetime.now() - start
print("Training completed in time: ", duration)

In [ ]:
test_accuracy=model.evaluate(x_test,y_test,verbose=0)
print((test_accuracy[1]))

0.9200000166893005


**testing**

In [ ]:
filename="/content/drive/MyDrive/project_dataset/audio/shankarabharanam/Alliyaambal.mp3"
audio, sample_rate = librosa.load(filename, res_type='kaiser_fast') 
mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
print(mfccs_scaled_features)
mfccs_scaled_features=mfccs_scaled_features.reshape(1,-1)
print(mfccs_scaled_features)
print(mfccs_scaled_features.shape)
predicted_label=model.predict_classes(mfccs_scaled_features)
print(predicted_label)
prediction_class = labelencoder.inverse_transform(predicted_label) 
prediction_class

[-214.13089     126.184814    -50.747826     13.556474     25.185953
  -16.20438      -8.365451      4.6538396   -21.921946      2.9007835
  -14.306373    -10.43093      -9.851578     -9.621131     -9.2385645
   -5.3910894   -10.510973     -9.745995    -14.947873    -10.400327
   -9.668875     -1.4432622    -0.62414277   -2.6759596    -6.7389
  -11.819227    -10.886211     -8.154639     -2.4416199    -1.6975592
   -4.155437     -6.6122184    -3.0241554    -3.3031032    -2.4376962
   -4.4701366    -2.2615328    -1.2353498    -1.9227601    -6.8753347 ]
[[-214.13089     126.184814    -50.747826     13.556474     25.185953
   -16.20438      -8.365451      4.6538396   -21.921946      2.9007835
   -14.306373    -10.43093      -9.851578     -9.621131     -9.2385645
    -5.3910894   -10.510973     -9.745995    -14.947873    -10.400327
    -9.668875     -1.4432622    -0.62414277   -2.6759596    -6.7389
   -11.819227    -10.886211     -8.154639     -2.4416199    -1.6975592
    -4.155437     -6.6

/usr/local/lib/python3.10/dist-packages/tensorflow/python/keras/engine/sequential.py:454: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array(['shankarabharanam'], dtype='<U16')

## CNN

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical

# Perform label encoding and convert labels to one-hot encoding
label_encoder = LabelEncoder()
b = to_categorical(label_encoder.fit_transform(y))

# Split the dataset into training and testing sets
a_train, a_test, b_train, b_test = train_test_split(a, b, test_size=0.2, random_state=0)

# Define the CNN model
model = Sequential()
model.add(Conv1D(32, kernel_size=3, activation='relu', 
                 input_shape=(a_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.25))
model.add(Conv1D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(b.shape[1], activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Train the model
epochs=15
history=model.fit(a_train, b_train, batch_size=32, 
                  epochs=epochs, validation_data=(a_test, b_test))
# Save the model
model.save('cnn_model.h5')

In [ ]:
model.summary()

In [ ]:
accuracy = history.history['accuracy']
ac=accuracy[epochs-1]
print("Accuracy:", ac)

Accuracy: 0.837837815284729


In [ ]:
# Preprocess the custom input (extract features)
filename = '/content/drive/MyDrive/final project dataset/audio/chakravaakam/pamba14.mp3'
mfccfeature = features_extractor(filename)

# Prepare the input for prediction
mfccfeature = np.array([mfccfeature])
mfccfeature = np.reshape(mfccfeature, (mfccfeature.shape[0], mfccfeature.shape[1], 1))

# Perform the prediction
predictions = model.predict(mfccfeature)

# Convert the predictions to class labels
predicted_labels = np.argmax(predictions, axis=1)
print(predicted_labels)
predicted_class = label_encoder.inverse_transform(predicted_labels)

print("Predicted class:", predicted_class)


1/1 [==============================] - 0s 33ms/step
[2]
Predicted class: shagarabharanam
